# name/variable_scope 的作用
@author: huangyongye<br/>
@creat_date: 2017-03-08<br/><br/>
refer to: https://www.tensorflow.org/programmers_guide/variable_scope <br/>
** 起因：在运行 RNN LSTM 实例代码的时候出现 ValueError。 ** <br/>
在 TensorFlow 中，经常会看到这两个东东出现，这到底是什么鬼，是用来干嘛的。在做 LSTM 的时候遇到了下面的错误：<br/>
 <font color='red'>ValueError: Variable rnn/basic_lstm_cell/weights already exists, disallowed.</font><br/>
然后谷歌百度都查了一遍，结果也不知是咋回事。我是在 jupyter notebook 运行的示例程序，第一次运行的时候没错，然后就总是出现上面的错误。后来才知道是 get_variable() 和 variable_scope() 搞的鬼。 <br/>
下面就来分析一下 TensorFlow 中到底用这来干啥。

In [1]:
import tensorflow as tf

In [2]:
# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [4]:
tf.Variable?

## 1. 首先看看比较简单的 tf.name_scope('scope_name'). 

tf.name_scope 主要结合 tf.Variable() 来使用，方便参数命名管理。

In [3]:
'''
Signature: tf.name_scope(*args, **kwds)
Docstring:
Returns a context manager for use when defining a Python op.
'''
# 也就是说，它的主要目的是为了更加方便地管理参数命名。
# 与 tf.Variable() 结合使用。简化了命名
with tf.name_scope('conv1') as scope:
    weights1 = tf.Variable([1.0, 2.0], name='weights')
    bias1 = tf.Variable([0.3], name='bias')

# 下面是在另外一个命名空间来定义变量的
with tf.name_scope('conv2') as scope:
    weights2 = tf.Variable([4.0, 2.0], name='weights')
    bias2 = tf.Variable([0.33], name='bias')

# 所以，实际上weights1 和 weights2 这两个引用名指向了不同的空间，不会冲突
print weights1.name
print weights2.name

conv1/weights:0
conv2/weights:0


In [4]:
# 注意，这里的 with 和 python 中其他的 with 是不一样的
# 执行完 with 里边的语句之后，这个 conv1/ 和 conv2/ 空间还是在内存中的。这时候如果再次执行上面的代码
# 就会再生成其他命名空间
with tf.name_scope('conv1') as scope:
    weights1 = tf.Variable([1.0, 2.0], name='weights')
    bias1 = tf.Variable([0.3], name='bias')

with tf.name_scope('conv2') as scope:
    weights2 = tf.Variable([4.0, 2.0], name='weights')
    bias2 = tf.Variable([0.33], name='bias')

print weights1.name
print weights2.name

conv1_1/weights:0
conv2_1/weights:0


In [1]:
import tensorflow as tf

## 2.下面来看看 tf.variable_scope('scope_name') 

tf.variable_scope() 主要结合 tf.get_variable() 来使用，实现 变量共享。

In [1]:
# 这里是正确的打开方式~~~可以看出，name 参数才是对象的唯一标识
import tensorflow as tf
with tf.variable_scope('v_scope') as scope1:
    Weights1 = tf.get_variable('Weights', shape=[2,3])
    bias1 = tf.get_variable('bias', shape=[3])
    
# 下面来共享上面已经定义好的变量
# note: 在下面的 scope 中的变量必须已经定义过了，才能设置 reuse=True，否则会报错
with tf.variable_scope('v_scope', reuse=True) as scope2:
    Weights2 = tf.get_variable('Weights')

print Weights1.name
print Weights2.name
# 可以看到这两个引用名称指向的是同一个内存对象

v_scope/Weights:0
v_scope/Weights:0


也可以结合 tf.Variable() 一块使用。

In [2]:
import tensorflow as tf
# 注意， bias1 的定义方式
with tf.variable_scope('v_scope') as scope1:
    Weights1 = tf.get_variable('Weights', shape=[2,3])
#     bias1 = tf.Variable([0.52], name='bias')
    
# 下面来共享上面已经定义好的变量
# note: 在下面的 scope 中的get_variable()变量必须已经定义过了，才能设置 reuse=True，否则会报错
with tf.variable_scope('v_scope', reuse=True) as scope2:
    Weights2 = tf.get_variable('Weights')
    bias2 = tf.Variable([0.52], name='bias')

print Weights1.name
print Weights2.name
print bias2.name

v_scope/Weights:0
v_scope/Weights:0
v_scope_1/bias:0


如果 reuse=True 的scope中的变量没有已经定义，会报错！！

In [1]:
import tensorflow as tf
# 注意， bias1 的定义方式
with tf.variable_scope('v_scope') as scope1:
    Weights1 = tf.get_variable('Weights', shape=[2,3])
    bias1 = tf.Variable([0.52], name='bias')
    
print Weights1.name
print bias1.name
    
# 下面来共享上面已经定义好的变量
# note: 在下面的 scope 中的get_variable()变量必须已经定义过了，才能设置 reuse=True，否则会报错
with tf.variable_scope('v_scope', reuse=True) as scope2:
    Weights2 = tf.get_variable('Weights')
    bias2 = tf.get_variable('bias', [1])  # ‘bias

print Weights2.name
print bias2.name

# 这样子的话就会报错
# Variable v_scope/bias does not exist, or was not created with tf.get_variable()

v_scope/Weights:0
v_scope/bias:0


## 3. RNN/LSTM 常常这样用

refer to: https://morvanzhou.github.io/tutorials/machine-learning/tensorflow/5-12-scope/ <br/>
RNN 和普通的网络是不太一样的，主要区别就是很多情况下，<font color="red">训练网络和测试网络结构不一样</font>，我们用 train_rnn 来指训练网络，用 test_rnn 来指测试网络。<br/>
在官方给的利用 LSTM 来对 MNIST 字符分类的例子中，train_rnn 和 test_rnn 是一样的。我们都是通过输入 28 个time_steps 之后输出它的类别，无论是训练还是测试的时候都是这样。<br/>
但是，在 char_rnn 语言模型中:
- 训练时候，我们输入<font color="red">40个字符</font>，然后输出一个字符，再来计算 loss。 <br/>
- 而当模型训练好了之后，我们都是每看到<font color="red"> 1 个新的字符</font>,我们就会对下一个字符做预测。<br/>
所以会有 train_rnn 和 test_rnn 两个不同的结构。但是我们的 test_rnn 只是输出的频率不一样而已（每一步都输出），它所用到的所有参数都是在 train_rnn 中已经训练好的。所以，**由于结构不同，而所需要的参数相同**，我们就可以，或者说就必须要用到变量共享的机制了。下面是我们使用的方法。

In [ ]:
with tf.variable_scope('rnn') as scope:
    sess = tf.Session()
    train_rnn = RNN(train_config)
    # 声明变量共享
    scope.reuse_variables()
    test_rnn = RNN(test_config)
    sess.run(tf.global_variables_initializer())